# Laboratorio 3

-  [Mónica Salvatierra - 22249](https://github.com/alee2602)
- [Paula Barillas - 22764](https://github.com/paulabaal12)
- [Derek Arreaga - 22537](https://github.com/FabianKel)

#####  LINK DE REPOSITORIO
https://github.com/FabianKel/LAB3-IA

## Task 1 - Preguntas Teóricas

1. Explique la diferencia entre descenso de gradiente, descenso de gradiente por mini batches y descenso de
gradiente estocástico. Asegúrese de mencionar las ventajas y desventajas de cada enfoque.

2. Compare y contraste técnicas de extracción de features (feature extraction) y selección de features (feature
selection) en machine learning. De ejemplos de escenarios donde cada técnica sería más apropiada.

3. Describa la arquitectura y el funcionamiento de un perceptrón de una sola capa (un tipo de red neuronal sin
backpropagation). Explique cómo aprende y la forma en la que actualiza sus parámetros.

## Task 2 - Ejercicios Prácticos

### Task 2.1 - Gradiente Descendiente Estocástico

Implemente el descenso de gradiente estocástico, el descenso de gradiente y el descenso de gradiente por mini
batches para una función polinómica de grado 3. Luego, grafique la función aproximada por cada uno de los métodos
solicitados y la distribución real de puntos. Para esto considere que:
* El polinomio que debe usar es 2 * x^3 - 3 * x^2 + 5 * x + 3
* Considere el uso de ```np.polyval```
* Considere el uso de ```np.polyder```
* Compare el tiempo de ejecución de cada uno de los métodos solicitados. De esto, mencione cuál método
fue más rápido y por qué.
* Compare el fitness de cada método solicitado (recuerde hacer una evaluación numérica y no sólo visual). De
esto mencione cuál método lo hizo mejor y por qué

### **Importar las librerías a utilizar** 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from sklearn.model_selection import train_test_split

%matplotlib inline


### **Definir funciones base**

In [ ]:

# 1. Definir el polinomio y su derivada
coefs = [2, -3, 5, 3] 
poly = np.poly1d(coefs)
deriv_poly = np.polyder(poly)

### **Generar datos**

In [165]:
# 2. Generación de datos
np.random.seed(1316)
x_data = np.linspace(-2, 2, 150)
y_data = poly(x_data) + np.random.normal(0, 2, size=x_data.shape) 

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1316)

### Task 2.2 - Feature Selection

Tome de nuevo el dataset del laboratorio pasado referente a la clasificación de Partidas de League of Legends.

Sobre este aplique al menos 3 técnicas de feature selection distintas, y con el dataset resultante vuelva a ajustar el modelo de Support Vector Machine (de la librería Sklearn).

Recuerde que:
* Debe definir una métrica de desempeño y justificar el por qué de esta métrica
* Mida el performance de su modelo con las tres técnicas de feature selection y compare cada una de ellas. Es decir, debe tener al menos 4 valores a comparar (el obtenido en el laboratorio pasado, y al menos 3 de las técnicas de feature selection)
* De las medidas obtenidas, diga qué versión lo hizo mejor y por qué.
* Recuerde que para hacer parameter tuning, deben usar las mismas columnas seleccionadas después del
feature selection
* Recuerde que su variable objetivo es “blueWins”

### Task 2.3 - Perceptrón

Implemente un perceptrón de una sola capa para realizar una clasificación binaria del dataset de Iris(incluido en ```sklearn.datasets.load_iris```). Visualice la frontera de decisión que fue aprendida por el perceptrón.
Recuerde que:
* Debe cargar el dataset de Iris desde la librería mencionada
* Use solamente dos features, siendo estos sepal length y sepal width
* Asegúrese de que su clase Perceptron tenga los métodos de fit y predict
* Visualice la frontera de decisión
    * El eje Y debería ser sepal width y el eje X debería ser sepal length
* Debe definir una métrica de desempeño y justificar el por qué de esta métrica. Con dicha métrica, evalúe el rendimiento de su modelo. Es decir, diga si lo hizo bien o no y el por qué.